In [ ]:
# model_api.py
from flask import Flask, request, jsonify
  # importer ton modèle ici

app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    input_data = data['input']
    input_array = np.array(input_data, dtype=np.float32)

    horizon = 10
    preds = predict_horizon("bilstm_model.h5", input_array, horizon, scaler_path='scaler.pkl')


    # Supposons que ton modèle renvoie une séquence

    return jsonify({'sequence': preds.tolist() })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.104:5000
Press CTRL+C to quit


In [ ]:
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.optimizers import Adam


def predict_horizon(model_path, seed_sequence, horizon, scaler_path='scaler.pkl'):
    # 1) Load model
    # 1) Load the model without trying to resolve the old metric
    model = load_model(model_path, compile=False)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    # 2) Load scaler
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)

    # 3) Scale the seed_sequence
    # seed_sequence: shape (seq_len, n_features)
    seed_scaled = scaler.transform(seed_sequence)

    seq_len, n_features = seed_sequence.shape
    buffer = seed_scaled.copy()

    preds_scaled = []

    for _ in range(horizon):
        x = buffer.reshape((1, seq_len, n_features))
        yhat = model.predict(x, verbose=0)[0]   # shape = (n_features,)
        preds_scaled.append(yhat)
        buffer = np.vstack([buffer[1:], yhat])

    preds_scaled = np.array(preds_scaled)       # shape = (horizon, n_features)

    # 4) Inverse-transform predictions
    preds = scaler.inverse_transform(preds_scaled)
    return preds



In [ ]:
if __name__ == "__main__":
    # Suppose your sequence length is 5, features=2 (lat, lon)
    seed = np.array([
        [57, 1399],
        [59, 1393],
        [63, 1387],
        [67, 1380],
        [72, 1373],
    ], dtype='float32')

    future_steps = 10
    preds = predict_horizon("bilstm_model.h5", seed, future_steps)
    print("Predicted next points:\n", preds)

D:\python\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
D:\python\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Predicted next points:
 [[  77.78668  1365.6927  ]
 [  84.0671   1358.181   ]
 [  90.64189  1350.495   ]
 [  97.18059  1342.7139  ]
 [ 103.438965 1334.9149  ]
 [ 109.43704  1327.1766  ]
 [ 115.21652  1319.5929  ]
 [ 120.794106 1312.2361  ]
 [ 126.18649  1305.1686  ]
 [ 131.4094   1298.449   ]]
